In [4]:
import bert
import glob
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from bert import run_classifier
from bert import optimization
from bert import tokenization
from datetime import datetime


import tools as tl
import score_predictions as verify

ImportError: cannot import name 'abs'

In [2]:
train_dataset = [list(tl.read_train_dataset(review_file))[0] for review_file in glob.glob('case_data/case_data/train/*.txt')]
reviews_train = pd.DataFrame(train_dataset)
reviews_train.review_score = reviews_train.review_score.apply(lambda x: int(x)/10)
reviews_train.head()

,review,review_score,review_source,review_title
0,This movie is a big letdown. Here you have a g...,0.4,reviews6990.txt,Not good enough
1,I gave this movie such a high mark because it ...,0.9,reviews794.txt,Funny and well acted without a story
2,"IMO this movie is not ""Catholic Indoctrination...",0.5,reviews3999.txt,"Faith,Fright and Fear Night"
3,DALLAS BUYERS CLUB (2013) ***1/2 Matthew McCon...,0.9,reviews8743.txt,JKL: The Movie - McConaughey gives an Oscar-wo...
4,While this movie's characters and plot have po...,0.1,reviews3761.txt,No substance to grab you with


In [4]:
reviews_train['text'] = [[reviews_train.review_title.iloc[w]+', '+reviews_train.review.iloc[w]][0] for w in range(len(reviews_train))]

In [5]:
test_dataset = [list(tl.read_test_dataset(review_file))[0] for review_file in glob.glob('case_data/case_data/test/*.txt')]
reviews_test = pd.DataFrame(test_dataset)
reviews_test.head()

,review,review_source,review_title
0,Ignore all the political diatribes from the ha...,reviews794.txt,Superb Psychological Drama/Character Study/Pol...
1,"OK, so far I've only seen everyone praising th...",reviews421.txt,Non-existing baby hurting itself!
2,When I first saw the previews for Guy Ritche's...,reviews1167.txt,The Man from UNCLE is a spy comedy that Hammer...
3,A great example of how didactism bores rather ...,reviews527.txt,Painfully Didactic & Political
4,"At first, when I decided to try watch this kin...",reviews1895.txt,Unexpected to be good!


In [6]:
reviews_test['text'] = [[reviews_test.review_title.iloc[w]+', '+reviews_test.review.iloc[w]][0] for w in range(len(reviews_test))]

In [7]:
reviews_train.head()

,review,review_score,review_source,review_title,text
0,This movie is a big letdown. Here you have a g...,0.4,reviews6990.txt,Not good enough,"Not good enough, This movie is a big letdown. ..."
1,I gave this movie such a high mark because it ...,0.9,reviews794.txt,Funny and well acted without a story,"Funny and well acted without a story, I gave t..."
2,"IMO this movie is not ""Catholic Indoctrination...",0.5,reviews3999.txt,"Faith,Fright and Fear Night","Faith,Fright and Fear Night, IMO this movie is..."
3,DALLAS BUYERS CLUB (2013) ***1/2 Matthew McCon...,0.9,reviews8743.txt,JKL: The Movie - McConaughey gives an Oscar-wo...,JKL: The Movie - McConaughey gives an Oscar-wo...
4,While this movie's characters and plot have po...,0.1,reviews3761.txt,No substance to grab you with,"No substance to grab you with, While this movi..."


In [8]:
DATA_COLUMN_REVIEW_TITLE = ['review_title']
DATA_COLUMN_REVIEW = ['review']
DATA_COLUMN = ['text']
LABEL_COLUMN = 'review_score'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = reviews_train.review_score.unique()

In [9]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = reviews_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN].values[0], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = reviews_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN].values[0], 
                                                                   text_b = None), axis = 1)

In [48]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples_title = reviews_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN_REVIEW_TITLE].values[0], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_title = reviews_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN_REVIEW_TITLE].values[0], 
                                                                   text_b = None), axis = 1)

In [49]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples_review = reviews_train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN_REVIEW].values[0], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_review = reviews_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN_REVIEW].values[0], 
                                                                   text_b = None), axis = 1)

In [50]:
train_InputExamples = [train_InputExamples_review] + [train_InputExamples_title]
test_InputExamples = [test_InputExamples_review] + [test_InputExamples_title]

In [10]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

W0910 18:19:57.774669 139786100320000 deprecation_wrapper.py:119] From /home/jupyter/notebooks/Casta/project/project/lib/python3.5/site-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [ ]:
# We'll set sequences to be at mo st 128 tokens long.
MAX_SEQ_LENGTH = 256
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples[0].values, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples[0].values, [None], MAX_SEQ_LENGTH, tokenizer)

In [12]:
# We'll set sequences to be at mo st 128 tokens long.
MAX_SEQ_LENGTH = 256
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, [None], MAX_SEQ_LENGTH, tokenizer)

W0910 18:20:56.331589 139786100320000 deprecation_wrapper.py:119] From /home/jupyter/notebooks/Casta/project/project/lib/python3.5/site-packages/bert/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



In [13]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [14]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        mae = tf.metrics.mean_absolute_error(label_ids,predicted_labels)
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "mae": mae,
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [15]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
OUTPUT_DIR = !pwd
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [16]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [17]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR[0]+'/output/',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [18]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


In [19]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [20]:
print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

W0910 18:24:13.793657 139786100320000 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Beginning Training!


W0910 18:24:26.652485 139786100320000 deprecation.py:506] From <ipython-input-13-ca03218f28a6>:34: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0910 18:24:26.686805 139786100320000 deprecation_wrapper.py:119] From /home/jupyter/notebooks/Casta/project/project/lib/python3.5/site-packages/bert/optimization.py:27: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.

W0910 18:24:26.703346 139786100320000 deprecation_wrapper.py:119] From /home/jupyter/notebooks/Casta/project/project/lib/python3.5/site-packages/bert/optimization.py:32: The name tf.train.polynomial_decay is deprecated. Please use tf.compat.v1.train.polynomial_decay instead.

W0910 18:24:26.709274 139786100320000 deprecation.py:323] From /usr/local/lib/python3.5/dist

Training took time  0:09:52.119491


In [21]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

In [22]:
def getPrediction(in_reviews):
    labels = reviews_train.review_score.unique()
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 1) for x in in_reviews] # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    #return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_reviews, predictions)]
    #yield [(sentence, labels[prediction['labels']]) for sentence, prediction in zip(in_reviews, predictions)]
    #yield predictions
    yield [labels[prediction['labels']] for prediction in predictions]

In [23]:
import time
start = time.time()
reviews_predictions = [list(w) for w in getPrediction(reviews_test.review.values)][0]
time.time()-start

W0910 18:35:20.209295 139786100320000 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


32.21771860122681

In [77]:
import time
start = time.time()
reviews_predictions_titles = [list(w) for w in getPrediction(reviews_test.review_title.values)][0]
time.time()-start

18.930612564086914

In [24]:
reviews_results = [int(float(w)*10) for w in reviews_predictions]
len(reviews_results)

2000

In [32]:
verify.score(reviews_results)

Please input the API url:  https://kogm9aixa5.execute-api.eu-central-1.amazonaws.com/dev/score
Please input your given api key:  JeAhRZDgVN6gEOtclQpuc4lVYnQtMnuC226QvJIw


Successfully saved api key to {API_INFO_PATH}. Please delete/edit this file if you gave the wrong key.


APIException: {"errorMessage": "An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.", "errorType": "NoSuchKey", "stackTrace": [["/var/task/score_predictions_lambda.py", 100, "score", "real = _read_real_labels()"], ["/var/task/score_predictions_lambda.py", 39, "_read_real_labels", "obj_body = obj.get()['Body'].read().decode('utf8')"], ["/var/runtime/boto3/resources/factory.py", 520, "do_action", "response = action(self, *args, **kwargs)"], ["/var/runtime/boto3/resources/action.py", 83, "__call__", "response = getattr(parent.meta.client, operation_name)(**params)"], ["/var/runtime/botocore/client.py", 314, "_api_call", "return self._make_api_call(operation_name, kwargs)"], ["/var/runtime/botocore/client.py", 612, "_make_api_call", "raise error_class(parsed_response, operation_name)"]]}

In [79]:
reviews_predictions_titles = [int(float(w)*10) for w in reviews_predictions_titles]
len(reviews_predictions_titles)

2000

In [81]:
verify.score(reviews_predictions_titles)

4.291

In [28]:
import os
os.environ["API_INFO_PATH"] = "JeAhRZDgVN6gEOtclQpuc4lVYnQtMnuC226QvJIw"